In [4]:
import numpy as np
import pandas as pd
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import keras
import matplotlib as plt
import seaborn as sns
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data = pd.read_csv('/content/drive/MyDrive/UnB/IEEE Computational Intelligence Society, UnB/Trainee/Dataset/Periodo 4/Musical_instruments_reviews.csv')

In [7]:
print(data.shape)

(10261, 9)


In [8]:
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB


In [10]:
data.isnull().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

In [11]:
data_proc = data.copy()
data_proc=data_proc.drop(['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime'], axis=1)
data_proc['textos']  = data_proc['summary']+ ', ' +data_proc['reviewText']
data_proc=data_proc.drop(['reviewText', 'summary'], axis=1)
data_proc.head()

,overall,textos
0,5.0,"good, Not much to write about here, but it doe..."
1,5.0,"Jake, The product does exactly as it should an..."
2,5.0,"It Does The Job Well, The primary job of this ..."
3,5.0,"GOOD WINDSCREEN FOR THE MONEY, Nice windscreen..."
4,5.0,"No more pops when I record my vocals., This po..."


In [12]:
data_proc['textos']

0        good, Not much to write about here, but it doe...
1        Jake, The product does exactly as it should an...
2        It Does The Job Well, The primary job of this ...
3        GOOD WINDSCREEN FOR THE MONEY, Nice windscreen...
4        No more pops when I record my vocals., This po...
                               ...                        
10256    Five Stars, Great, just as expected.  Thank to...
10257    Long life, and for some players, a good econom...
10258    Good for coated., I have tried coated strings ...
10259    Taylor Made, Well, MADE by Elixir and DEVELOPE...
10260    These strings are really quite good, but I wou...
Name: textos, Length: 10261, dtype: object

In [13]:
data_proc['overall'].value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

In [14]:
def f_sentimento(linha):
    if linha['overall'] > 3.0:
        veredito = 1
    else:
        veredito = 0
    return veredito

In [15]:
data_proc['sentimento'] = data_proc.apply(f_sentimento, axis=1)
data_proc.head()

,overall,textos,sentimento
0,5.0,"good, Not much to write about here, but it doe...",1
1,5.0,"Jake, The product does exactly as it should an...",1
2,5.0,"It Does The Job Well, The primary job of this ...",1
3,5.0,"GOOD WINDSCREEN FOR THE MONEY, Nice windscreen...",1
4,5.0,"No more pops when I record my vocals., This po...",1


In [16]:
data_proc['sentimento'].value_counts()

1    9022
0    1239
Name: sentimento, dtype: int64

In [17]:
def limpeza(texto):
    texto = str(texto).lower()
    texto = re.sub('\[.*?\]', '', texto)
    texto = re.sub('<.*?>+', '', texto)
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto)
    texto = re.sub('\n', '', texto)
    texto = re.sub('\w*\d\w*', '', texto)
    return texto

In [18]:
data_proc['textos']=data_proc['textos'].apply(lambda x:limpeza(x))
data_proc.head()

,overall,textos,sentimento
0,5.0,good not much to write about here but it does ...,1
1,5.0,jake the product does exactly as it should and...,1
2,5.0,it does the job well the primary job of this d...,1
3,5.0,good windscreen for the money nice windscreen ...,1
4,5.0,no more pops when i record my vocals this pop ...,1


In [19]:
data_proc['textos'][0]

'good not much to write about here but it does exactly what its supposed to filters out the pop sounds now my recordings are much more crisp it is one of the lowest prices pop filters on amazon so might as well buy it they honestly work the same despite their pricing'

In [20]:
corpus = []

for i in range(0, data.shape[0]):
    review = re.sub('[^a-zA-Z]',' ', str(data_proc['textos'][i]))
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
print(corpus[0])

good much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price


In [22]:
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(corpus).toarray()
y = data_proc['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [23]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [24]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
acuracia = round(accuracy_score(y_test, y_pred) * 100, 2)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
relatorio = classification_report(y_test, y_pred)


print('\n RMSE: {0}'.format(rmse))
print('\n Relatório:\n')
print(relatorio)

[[ 100  128]
 [ 274 1551]]

 RMSE: 0.4425053765555454

 Relatório:

              precision    recall  f1-score   support

           0       0.27      0.44      0.33       228
           1       0.92      0.85      0.89      1825

    accuracy                           0.80      2053
   macro avg       0.60      0.64      0.61      2053
weighted avg       0.85      0.80      0.82      2053



# **LSTM**

In [46]:
modelo_LSTM = keras.models.Sequential([keras.layers.Embedding(3000, 32, input_length = X_train.shape[1]),
                                       keras.layers.LSTM(128,return_sequences = True),
                                       keras.layers.Dense(3,activation='softmax'),
                                       keras.layers.Flatten(),
                                       keras.layers.Dense(2, activation="sigmoid")])

print(modelo_LSTM.summary())



Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 1000, 32)          96000     
                                                                 
 lstm_17 (LSTM)              (None, 1000, 128)         82432     
                                                                 
 dense_19 (Dense)            (None, 1000, 3)           387       
                                                                 
 flatten_6 (Flatten)         (None, 3000)              0         
                                                                 
 dense_20 (Dense)            (None, 2)                 6002      
                                                                 
Total params: 184,821
Trainable params: 184,821
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
modelo_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train_dummies = pd.get_dummies(y_train).values

modelo_LSTM.fit(X_train, y_train_dummies , epochs=3, batch_size=32)


Epoch 1/3
257/257 [==============================] - 490s 2s/step - loss: 0.3932 - accuracy: 0.8768
Epoch 2/3
257/257 [==============================] - 492s 2s/step - loss: 0.4060 - accuracy: 0.8754
Epoch 3/3
65/65 [==============================] - 42s 636ms/step


In [49]:
y_pred_LSTM = modelo_LSTM.predict(X_test)

65/65 [==============================] - 39s 597ms/step


In [58]:
y_pred_LSTM

array([[0.19315664, 0.6694587 ],
       [0.19172929, 0.6582888 ],
       [0.21127765, 0.6605045 ],
       ...,
       [0.2532833 , 0.6270453 ],
       [0.18729757, 0.67748785],
       [0.20207848, 0.6624735 ]], dtype=float32)

In [59]:
y_pred

array([1, 0, 1, ..., 0, 1, 1])

In [63]:
for i in range(len(y_pred_LSTM)):
  if y_pred_LSTM[i][0] > y_pred_LSTM[i][1] :
    y_pred_LSTM[i] = int(0)
  else:
    y_pred_LSTM[i] = int(1)

In [64]:
y_pred_LSTM

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       ...,
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)

In [53]:
np.rint(y_pred_LSTM)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [61]:
cm = confusion_matrix(y_test, y_pred_LSTM)
print(cm)
acuracia = round(accuracy_score(y_test, y_pred_LSTM) * 100, 2)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_LSTM))
relatorio = classification_report(y_test, y_pred_LSTM)


print('\n RMSE: {0}'.format(rmse))
print('\n Relatório:\n')
print(relatorio)

ValueError: ignored